In [ ]:
import os #여기부터 실행하세요. one drive에서 파일 다운받아두세요. 
import re
import openpyxl
import calendar
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time

cal = calendar.Calendar()

query = "여성"
query2 = "#2_" + query
year = 2019
xlist = list()

for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            dd = str(year) + str(month).zfill(2) + str(day).zfill(2)
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#2_html2text', query2, '#2_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#2_"+dd+"_"+query))
            xlist.append(xlsx_name)

In [ ]:
def replace_sub(text):
                
        from nltk.tokenize import regexp_tokenize
        from string import punctuation
        
        tlist, ttlist= list(), list()
        text = regexp_tokenize(text, r'[^\s\.][^\.\n]+')
        for _ in text:
            if len(_) > 11:
                tlist.append(_)
                
        text = ".\n".join(tlist) + "."

        text = re.sub(r'\d+\.(\n)\d+', r'', text)
        text = re.sub(r'\b(\w|[.])+@(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'\bhttps?://\w+(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-하-ㅣ\.?!\n ]', ' ', text)
        text = re.sub(r'\b[a-z][A-Za-z0-9]+\b', ' ', text)
        text = re.sub(r'\s{2,}', ' ', text) 
        text = re.sub(r'[\(\[].*?[\)\]]', '', text)
        text = re.sub('【.*】','',text)
        text = re.sub(r'"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\/:;<=>\@\\[\\\\\\]\\^_`\\{\\|\\}\\~', '', text)
        
        # 여기부터 새로운 태그 추가 #신문사 이름은 나중에 한꺼번에 뺄 것이니 추가하지 말 것
        text = re.sub(r"-\d+-", '', text) # 이거 전에 '-'제거하면 안됨.
        text = re.sub(r"\b\w{3}\s기자", '', text)
        text = re.sub(r'\s?[a-z]\s', ' ', text)
        text = re.sub(r'\s+\d+px?', '', text)
        text = re.sub(r'\s\d+\s\d+\s\d+\s', '', text)
        text = re.sub(r'\s?(\d+\s)', '', text)
        text = re.sub(r'((\d+[a-z])+)', '', text)
        text = re.sub(r'[A-Z0-9a-z][A-Z0-9a-z]', '', text)
        text = re.sub(r'\b[A-Z0-9]\b', '', text)

        
        text = text.replace('출처', '').replace('방송캡처', '').replace('-', '').replace('오류를 우회하기 위한 함수 추가.', '')
        text = text.replace('동영상 뉴스.', '').replace('아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', '')
        text = text.replace(' 사진 ', ' ').replace('경남신문의 콘텐츠는 저작권법의 보호를 받는 바 무단전재 복사 재배포를 금합니다.', '')
        text = text.replace('돋움', '').replace('메인이미지', '').replace('. p .', "\n").replace('정상처리되지 않았습니다.', '')
        text = text.replace(' p ', '\n').replace('IMGFLOATING ', '').replace('캡쳐 ', '').replace(' P ', '').replace('INSERT RANDOM NUMBER HERE', '')
        text = text.replace('OptimaA Scrip Ta', '').replace('시원하게 털어드립니다.', '').replace('저작권자 인사이트 무단전재 및 재배포 금지 .', '')
        text = text.replace('수직배너 와이드', '').replace('www.', '').replace('동영상이 안 나오면', '').replace('여기를 누르시면 동영상을 별도의 새 창에서 볼 수 있습니다.', '')
        try: 
            text = text[:text.rindex('다.')+2]
            
            text = text.split(sep='.')
            for _ in text:
                if len(re.sub(r'\s{2,}', ' ', _)) > 11:
                    ttlist.append(re.sub(r'\s{2,}', ' ', _))
                    
            text = ".".join(ttlist) + "."
            
            return text
        except:
            return 0


In [ ]:
"""
유미 0-91 >>>>>>>>>> 0,100
지은 92-182 >>>>>>>>>>
은서 183-273 >>>>>>>>>>
유리 274-364 >>>>>>>>>>
"""

df = pd.read_excel(xlist[0]) # 여기에 숫자 바꿔가면서 하면 돼

n = 100
text = df.iloc[n, 3]
url = df.iloc[n, 4]
print(url)
print(replace_sub(text))
#re.findall(r'((\d+[a-z])+)', text)

In [ ]:
import requests
from bs4 import BeautifulSoup

tag = "div.cont_cont"

raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = BeautifulSoup(raw.text, "html.parser")
html.select(tag)